In [37]:
from bs4 import BeautifulSoup
import requests
import smtplib
import time
import datetime
import csv 
import pandas as pd
import re

In [48]:
def check_price():
    URL = 'https://www.amazon.es/Purina-Felix-Gelatina-comida-gatos/dp/B08WSYSQVL?pd_rd_w=pcWWJ&content-id=amzn1.sym.39a0530e-cdee-46be-8da1-bcc874e212f9&pf_rd_p=39a0530e-cdee-46be-8da1-bcc874e212f9&pf_rd_r=YHPFHF515DR4CBEPVKPB&pd_rd_wg=F6zqN&pd_rd_r=2386a2f0-048a-4b81-9c8b-cca0061ac417&pd_rd_i=B08WSYSQVL&psc=1&ref_=pd_bap_d_grid_rp_0_1_ec_i'

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36", 
               "Accept-Encoding":"gzip, deflate", 
               "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", 
               "DNT":"1",
               "Connection":"close", 
               "Upgrade-Insecure-Requests":"1"}

    page = requests.get(URL, headers=headers)

    soup1 = BeautifulSoup(page.content, "html.parser")

    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

    title = soup2.find(id='productTitle').get_text(strip=True)

    price_element = soup2.find(id="sns-base-price").get_text(strip=True)
    price = price_element.strip()[:-1]

    today = datetime.date.today()

    header = ['Title', 'Price', 'Date']
    data = [title, price, today]

    with open('AmzScrape_catfood.csv', 'a+', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)
    
    price_to_send = re.sub(r'[^\d.,]', '', price)
    price_to_send = price_to_send.replace(',', '.')
        
    price_to_send = float(price_to_send)   
    if price_to_send < 25:
        send_mail()

In [56]:
def send_mail():
    server = smtplib.SMTP_SSL('smtp.gmail.com',465)
    server.ehlo()
    server.ehlo()
    server.login('myemail','password')
    
    subject = "Olivia's favorite food is below 25! :O!"
    body = "Don't pay full price for Olivia's food! Link here: https://www.amazon.es/Purina-Felix-Gelatina-comida-gatos/dp/B08WSYSQVL?pd_rd_w=pcWWJ&content-id=amzn1.sym.39a0530e-cdee-46be-8da1-bcc874e212f9&pf_rd_p=39a0530e-cdee-46be-8da1-bcc874e212f9&pf_rd_r=YHPFHF515DR4CBEPVKPB&pd_rd_wg=F6zqN&pd_rd_r=2386a2f0-048a-4b81-9c8b-cca0061ac417&pd_rd_i=B08WSYSQVL&psc=1&ref_=pd_bap_d_grid_rp_0_1_ec_i"
   
    msg = f"Subject: {subject}\n\n{body}"
    
    server.sendmail(
        'myemail@gmail.com',
        'myemail@gmail.com',
        msg
     
    )

In [ ]:
while(True):
    check_price()
    time.sleep(86400)